#### Import section and global variables



In [3]:
from bs4 import BeautifulSoup
import requests
import re
import sys, os
import sqlite3
import csv
import time

# When an exception occurs, set this variable to terminate code execution.
errEvent = False

print("Cell Executed")

Cell Executed


#### Fetching the HTML source code of money.cnn.com/data/hotstocks website

In [4]:
# Use this header in requests to prevent the User Agent from being validated
h={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

try:
    if errEvent:
        raise KeyboardInterrupt
        
    html = requests.get('https://money.cnn.com/data/hotstocks/', headers=h)
    soup = BeautifulSoup(html.text, 'html.parser')
    print("Cell Executed")
except KeyboardInterrupt:
    print("Code execution interrupted\nError in the previous cell, check the output of the above cell\nOnce resolved Restart and runn all cells")
except Exception as e:
    errCond = True
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Message: ", exc_obj, "\nType: ", exc_type, "\nLine no.: ", exc_tb.tb_lineno)

Cell Executed


### We possess stock symbol, however that is in the format "F" (Ford Motor Company), but the google finance url format is "F:NYSE"

#### Because the symbol obtained from the preceding HTML document is not immediately applicable to a Google Finance query, we will acquire the document for each stock and use it to locate the appropriate symbol

In [5]:
try:
    if errEvent:
        raise KeyboardInterrupt
    
    active_stocks = soup.find('table', class_='wsod_dataTable wsod_dataTableBigAlt')
    active_stocks_dict = {}

    # Seeking a link to the stock's page
    for t in active_stocks.find_all('a', class_='wsod_symbol'):
        
        # Creating dictionary a symbol (key) and link (value)
        active_stocks_dict[t.text] = 'https://money.cnn.com/' + t.get('href')

    # Get the precise ticker symbol for a Google Finance search.
    for s, l in active_stocks_dict.items():
        
        # The document with the format 'F:NYSE' symbol is being retired
        _soup = BeautifulSoup(requests.get(l, headers=h).text, 'html.parser')
        s_t = _soup.find('span', class_='wsod_smallSubHeading')

        # Regular expression for eliminating '(' and ')', as well as dividing the data on ':' and flipping the sequence
        _sym = ':'.join(re.sub(r'[\(\)]', '', s_t.text).split(":")[::-1])
        active_stocks_dict[s] = 'https://www.google.com/finance/quote/' + _sym
    print("Cell Executed")
except KeyboardInterrupt:
    print("Code execution interrupted\nError in the previous cell, check the output of the above cell\nOnce resolved Restart and runn all cells")
except Exception as e:
    errCond = True
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Message: ", exc_obj, "\nType: ", exc_type, "\nLine no.: ", exc_tb.tb_lineno)

Cell Executed


#### The day opening price, average volume, and PE-ratio are collected from the Google Finance website


In [6]:
try:
    if errEvent:
        raise KeyboardInterrupt
        
    stocks_data = {}

    # On the finance page, the opening price, volume, and pe-ratio have been filtered
    for symbl, lnk in active_stocks_dict.items():
        _soup = BeautifulSoup(requests.get(lnk, headers=h).text, 'html.parser')

        # Our necessary data is included under div tags with the class P6K39c
        t = _soup.find_all('div', class_='P6K39c')
        
        # We do not have direct access to opening prices, but we do have access to day-range pricing, which we may use to establish opening prices
        p = float(t[1].text.split(' - ')[0].replace('$', ''))
        
        v = int(float(t[4].text.replace('M', '').replace(',', '')))

        # On the Google Finance page, the PE-ratio might be a number or a '-'
        if t[5].text != '-':
            pe = float(t[5].text)
        else:
            pe = -1
        stocks_data[symbl] = {"open_price": p,"volume": v,"pe_ratio": pe}
    print("Cell Executed")
except KeyboardInterrupt:
    print("Code execution interrupted\nError in the previous cell, check the output of the above cell\nOnce resolved Restart and runn all cells")
except Exception as e:
    errCond = True
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Message: ", exc_obj, "\nType: ", exc_type, "\nLine no.: ", exc_tb.tb_lineno)

Cell Executed


#### Performing file management, that is, identifying whether or not files exist and acting accordingly

In [8]:
try:
    if errEvent:
        raise KeyboardInterrupt
    
    # Bool variables to validate if files exisits or not
    text_exists = os.path.isfile('stocks.txt')
    db_exits = os.path.isfile('StocksDatabase.db')

    # If the files are present, then deleting them
    if text_exists:
        os.remove('stocks.txt')

    if db_exits:
        f = os.open('StocksDatabase.db', os.O_WRONLY)
        os.close(f)
        os.remove('StocksDatabase.db')

    txt_file = open('stocks.txt', 'w')
    con = sqlite3.connect('StocksDatabase.db')
    print("Cell Executed")
except KeyboardInterrupt:
    print("Code execution interrupted\nError in the previous cell, check the output of the above cell\nOnce resolved Restart and runn all cells")
except Exception as e:
    errCond = True
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Message: ", exc_obj, "\nType: ", exc_type, "\nLine no.: ", exc_tb.tb_lineno)

Cell Executed


#### Recording data in Text file and SQLite database

In [ ]:
try:
    if errEvent:
        raise KeyboardInterrupt
       
    cur = con.cursor()
    txt_file.write(','.join(['Symbol', 'Open Price', 'Volume', 'PE-Ratio']) + '\n')

    # Creating a 'StocksTable' table with a given structure
    cur.execute('''CREATE TABLE IF NOT EXISTS StocksTable
                   (TickerSymb TEXT, OpenPrice REAL, Volume INTEGER, PERatio REAL)''')
    
    
    for symbl, stats in stocks_data.items():
        data_row = ','.join([str(symbl), str(stats['open_price']), str(stats['volume']), str(stats['pe_ratio'])]) + '\n'
        txt_file.write(data_row)

        # Inserting data into the database
        cur.execute('''
            INSERT INTO StocksTable VALUES ('{}', {}, {}, {})
        '''.format(symbl, stats['open_price'], stats['volume'], stats['pe_ratio'])
        )

    else:
        con.commit()
        con.close()
        txt_file.close()
    print("Cell Executed")
except KeyboardInterrupt:
    print("Code execution interrupted\nError in the previous cell, check the output of the above cell\nOnce resolved Restart and runn all cells")
except Exception as e:
    errCond = True
    exc_type, exc_obj, exc_tb = sys.exc_info()
    print("Message: ", exc_obj, "\nType: ", exc_type, "\nLine no.: ", exc_tb.tb_lineno)  
    txt_file.close()
    print("If another error is thrown, restart the kernel")
    con.rollback()
    con.close()